In [73]:
%pip install groq pandas tabulate sqlparse

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import sqlite3
from tabulate import tabulate
from groq import Groq

groq_api_key = "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"  # Replace with your actual Groq API key
client = Groq(api_key=groq_api_key)

In [ ]:
# load files
df = pd.read_csv("data_2010.csv", sep=';')
meta_sheets = pd.read_excel("sorular-kodlama.xlsx", sheet_name=None)

# optimized metadata_text (token-safe)
sorular_df = meta_sheets["sorular"]
column_descriptions = []
for i, (_, row) in enumerate(sorular_df.iterrows()):
    if i >= 5:  # limit columns to 5 instead of 10
        break
    col = str(row["Variable"]).strip()
    label = str(row["Label"]).strip()
    measure = str(row["Measurement Level"]).strip()
    column_descriptions.append(f"- {col}: {label} ({measure})")

# Limit code mappings more aggressively
kodlama_df = meta_sheets["kodlama"]
code_mappings = {}
for _, row in kodlama_df.iterrows():
    var = str(row["Value"]).strip()
    code = str(row["x"]).strip()
    label = str(row["Label"]).strip()
    if var not in code_mappings:
        code_mappings[var] = []
    if len(code_mappings[var]) < 2:  # max 2 values per var instead of 3
        code_mappings[var].append(f"{code} → {label}")

metadata_text = "DATA COLUMNS:\n" + "\n".join(column_descriptions) + "\n\nCODE MAPPINGS:\n"
for var, mappings in code_mappings.items():
    metadata_text += f"{var}:\n" + "\n".join(mappings) + "\n...\n"

# print metadata length to debug
print(f"Metadata text length: {len(metadata_text)} characters")

# load Data into In-Memory SQLite
conn = sqlite3.connect(":memory:")
df.to_sql("data_2010", conn, index=False, if_exists="replace")

/var/folders/mz/gzyhrjkn1jd9t0hfvp2g3j_m0000gn/T/ipykernel_11730/2541751064.py:2: DtypeWarning: Columns (6,7,11,12,15,17,18,19,32,45,60,83,84,85,88,89,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,153,182,184,186,208,209,210,211,212,213,214,215,216,217,218,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data_2010.csv", sep=';')


Metadata text length: 15013 characters


25639

In [74]:
import sqlparse

def clean_sql_query(raw_sql):
    # Remove code fences if present
    if raw_sql.startswith("```") and raw_sql.endswith("```"):
        raw_sql = raw_sql.strip("`").strip()

    # Strip weird quote mistakes (common LLM issue)
    raw_sql = raw_sql.replace('COUNT(*)"', 'COUNT(*)').replace('COUNT(*) "', 'COUNT(*) ')

    # Format the SQL using sqlparse
    formatted_sql = sqlparse.format(
        raw_sql,
        reindent=True,
        keyword_case='upper'
    )

    return formatted_sql.strip()


In [ ]:
def ask_question(question: str):
    # turn question into SQL
    prompt_sql = f"""
    You are a data analyst. Given a user question and a table called 'data_2010', write a correct SQLite SQL query.
    Always wrap column names in double quotes — especially those containing periods or special characters.

    User question: "{question}"

    Here is documentation about the data columns and values:
    {metadata_text}

    Only return the SQL query, no explanation.
    """

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt_sql}],
        temperature=0.2
    )

    raw_sql_query = response.choices[0].message.content.strip()

    # Clean and format SQL
    sql_query = clean_sql_query(raw_sql_query)

    print("🔍 Generated SQL Query (cleaned and formatted):")
    print(sql_query)

    # run SQL on dataframe
    try:
        result_df = pd.read_sql_query(sql_query, conn)
    except Exception as e:
        return f"SQL Error: {e}\n\nHere’s the SQL I tried:\n{sql_query}"

    # summarize result back to user
    table_summary = result_df.head(10).to_markdown(index=False)
    prompt_answer = f"""
    You are a helpful data assistant. Based on the following question and SQL result, provide a short and clear answer.

    User question: "{question}"

    Query result:
    {table_summary}
    """

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt_answer}],
        temperature=0.3
    )

    return response.choices[0].message.content.strip()


In [70]:
def suggest_questions():
    prompt_suggest = f"""
    You are a data analyst. Given the following metadata, suggest 10 insightful and diverse questions that a user might ask to summarize or understand the dataset 'data_2010'. 
    Focus on providing questions related to counts, distributions, averages, trends, or interesting breakdowns.

    Metadata:
    {metadata_text}

    Only return the 10 questions in a numbered list, no explanations.
    """

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt_suggest}],
        temperature=0.3
    )

    suggestions = response.choices[0].message.content.strip()
    print("💡 Suggested Questions:")
    print(suggestions)


In [71]:
suggest_questions()

💡 Suggested Questions:
Here are 10 insightful and diverse questions that a user might ask to summarize or understand the dataset 'data_2010':

1. What is the distribution of respondents by age group?
2. How many respondents identify as female, and what is their average age?
4. What is the most common occupation among those who voted for the AKP?
5. What percentage of respondents have a high school diploma or higher?
6. How many respondents' average monthly income, and what is the distribution of income levels?
7. What is the most common response to the question about the importance of democracy?
8. How many people have a positive view of the AKP, and what is their average age?
9. What is the distribution of respondents by region, and how does this relate to their political party affiliation?
10. How many respondents have a university degree, and what is their average age?


In [94]:
print(ask_question("What is the most common job/profession?"))

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jw10264ket6vmges72m5hq8x` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 497487, Requested 4321. Please try again in 5m12.2594s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

How many men answered the questions?

How many records are there in 2010?

What is the most common job/profession?

How many people have the highest education level?

Show the number of people per profession.

How did the number of people starting work change between 2010 and 2011?

Which profession increased the most?

How many people that are students watch the channel with the code 2?